In [1]:
import csv
from pathlib import Path
import numpy as np
from ranking import Ranking
from index import Mode, InMemoryIndex
from encoder import TCTColBERTQueryEncoder as TCTColBERTQueryEncoderFF
# from pyserini.search.faiss import FaissSearcher, TctColBertQueryEncoder
from ir_measures import read_trec_qrels, calc_aggregate, nDCG, RR
import sys

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# in_memory_index = InMemoryIndex(mode=Mode.PASSAGE, encoder=TCTColBERTQueryEncoderFF("castorini/tct_colbert-msmarco"))
in_memory_index = InMemoryIndex.from_disk('../testing/ffindex_passage_2019_2020.pkl')
in_memory_index.encoder = TCTColBERTQueryEncoderFF("castorini/tct_colbert-msmarco")

Some weights of the model checkpoint at castorini/tct_colbert-msmarco were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
sparse_ranking_2019 = Ranking.from_file(Path("../testing/msmarco-passage-test2019-sparse10000.txt"))
sparse_ranking_2019.cut(5000)
sparse_ranking_2019.name = "sparse_ranking_2019"
sparse_ranking_2020 = Ranking.from_file(Path("../testing/msmarco-passage-test2020-sparse10000.txt"))
sparse_ranking_2020.cut(5000)
sparse_ranking_2020.name = "sparse_ranking_2020"
all_ids = set.union(
    *[set(sparse_ranking_2019[q_id].keys()) for q_id in sparse_ranking_2019.q_ids],
    *[set(sparse_ranking_2020[q_id].keys()) for q_id in sparse_ranking_2020.q_ids]
)
print(f"indexing {len(all_ids)} documents or passages")

indexing 440079 documents or passages


In [4]:
with open(
    "../testing/msmarco-test2019-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")

alpha = 0.2
result = in_memory_index.get_scores(
    sparse_ranking_2019,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False,
    normalise=False
)
result2 = in_memory_index.get_scores_rrf(
    sparse_ranking_2019,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False
)
result3 = in_memory_index.get_scores(
    sparse_ranking_2019,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False,
    normalise=True
)
qrels = list(read_trec_qrels("../testing/2019qrels-pass.txt"))
print(
    "BM25",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, sparse_ranking_2019.run)
)
print(
    f"Interpolation",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result[alpha].run)
)
print(
    f"RRF",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result2[alpha].run)
)
print(
    f"Interpolation w/ minimax",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result3[alpha].run)
)

loaded 200 queries
Encoding queries for interpolation...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 43/43 [00:00<00:00, 46.42it/s]


sparse_ranking_2019 None
Encoding queries for RRF...


100%|██████████| 43/43 [00:00<00:00, 78.25it/s]


sparse_ranking_2019 None
Encoding queries for interpolation...


100%|██████████| 43/43 [00:00<00:00, 81.03it/s]


sparse_ranking_2019 None
BM25 {RR(rel=2)@10: 0.7024178663713547, nDCG@10: 0.5058310024399072}
Interpolation {RR(rel=2)@10: 0.901937984496124, nDCG@10: 0.715806671562603}
RRF {RR(rel=2)@10: 0.8775193798449613, nDCG@10: 0.6816417471378418}
Interpolation w/ minimax {RR(rel=2)@10: 0.8949612403100775, nDCG@10: 0.722994260936128}


In [5]:
with open(
    "../testing/msmarco-test2020-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")

alpha = 0.2
result = in_memory_index.get_scores(
    sparse_ranking_2020,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False,
    normalise=False
)
result2 = in_memory_index.get_scores_rrf(
    sparse_ranking_2020,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False
)
result3 = in_memory_index.get_scores(
    sparse_ranking_2020,
    queries,
    alpha=alpha,
    cutoff=None,
    early_stopping=False,
    normalise=True
)
qrels = list(read_trec_qrels("../testing/2020qrels-pass.txt"))
print(
    "BM25",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, sparse_ranking_2020.run)
)
print(
    f"Interpolation",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result[alpha].run)
)
print(
    f"RRF",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result2[alpha].run)
)
print(
    f"Interpolation w/ minimax",
    calc_aggregate([nDCG@10, RR(rel=2)@10], qrels, result3[alpha].run)
)

loaded 200 queries
Encoding queries for interpolation...


100%|██████████| 54/54 [00:00<00:00, 77.78it/s]


sparse_ranking_2020 None
Encoding queries for RRF...


100%|██████████| 54/54 [00:00<00:00, 88.33it/s]


sparse_ranking_2020 None
BM25 {RR(rel=2)@10: 0.6554012345679013, nDCG@10: 0.4875508583120806}
Interpolation {RR(rel=2)@10: 0.797530864197531, nDCG@10: 0.671449718226886}
RRF {RR(rel=2)@10: 0.8151455026455027, nDCG@10: 0.6280697741206182}
